In [1]:
!git clone https://github.com/asahicantu/MIRNet.git
!pip install -q wandb
!pip install gdown
!pip install findpeaks
#!pip install -r requirements.txt

In [3]:
%cd /kaggle/working/MIRNet
from glob import glob
import os
from matplotlib import pyplot as plt
from mirnet.inference import Inferer
from mirnet.utils import download_dataset, plot_result
import noise


In [4]:
#download_dataset('LOL')

In [5]:
inferer = Inferer()
inferer.download_weights('1sUlRD5MTRKKGxtqyYDpTv7T3jOW6aVAL')
inferer.build_model(
    num_rrg=3, num_mrb=2, channels=64,
    weights_path='low_light_weights_best.h5'
)

In [107]:
import numpy as np
def noisify_speckle(image):
    gauss = None
    if len(image.shape) == 2:
        row, col = image.shape
        mu, sigma = 0, 0.7
        # gauss = np.random.normal(mu, sigma, row * col).astype('uint8')
        gauss = np.random.randn(row, col).astype('uint8')
        gauss = gauss.reshape(row, col)
    elif len(image.shape) == 3:
        row, col, ch = image.shape
        # gauss = np.random.randn(mu, sigma, row * col * ch)
        gauss = np.random.randn(row, col, ch).astype('uint8')
        gauss = gauss.reshape(row, col, ch) 
    noisy = image + image * gauss
    return noisy

def plot_results(original, noisy, enhanced):
    """Utility for Plotting inference result
    Args:
        image: original image
        enhanced: enhanced image
    """
    fig = plt.figure(figsize=(12, 12))
    fig.add_subplot(1, 3, 1).set_title('Original Image')
    _ = plt.imshow(original)
    fig.add_subplot(1, 3, 2).set_title('Noisy Image')
    _ = plt.imshow(noisy)
    fig.add_subplot(1, 3, 3).set_title('Enhanced Image')
    _ = plt.imshow(enhanced)
    
    plt.show()
    
    
    
def plot_hists(original, noisy, enhanced):
    fig = plt.figure(figsize=(12, 4))
    fig.add_subplot(1, 3, 1).set_title('Original Image')
    _ = plt.hist(np.array(original).ravel(),bins=256,range=(0,256))
    fig.add_subplot(1, 3, 2).set_title('Noisy Image')
    _ = plt.hist(np.array(noisy).ravel(),bins=256,range=(0,256))
    fig.add_subplot(1, 3, 3).set_title('Enhanced Image')
    enhanced = np.array(enhanced)
    np.array(img_denoise).shape
    _ = plt.hist(enhanced[:,:,0].ravel(),bins=256,range=(0,256))
    _ = plt.hist(enhanced[:,:,1].ravel(),bins=256,range=(0,256))
    _ = plt.hist(enhanced[:,:,2].ravel(),bins=256,range=(0,256))
    
    plt.show()
    
    
def signal_to_noise(a, axis=None, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    x = np.where(sd == 0, 0, m/sd)
    return round(float(str(x)), 5)


In [108]:
from PIL import Image
import cv2

img_cases = [
    'Fig0221(a)(ctskull-256).tif',
    'Fig0222(a)(face).tif',
    'Fig0507(a)(ckt-board-orig).tif',
    'Fig0519(a)(florida_satellite_original).tif'
]
results = []
##noise_imgs = glob('/kaggle/input/img-bw-noise/*')

for img_path in img_cases:
    img_path = os.path.join('/kaggle/input/d/asahicantu/img-bw-noise',img_path)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_noise = noisify_speckle(img)
    img_noise = cv2.cvtColor(img_noise,cv2.COLOR_GRAY2RGB)
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    
    im_pil = Image.fromarray(img_noise)
    noisy_image, img_denoise = inferer.infer_streamlit(im_pil)
    plot_results(img, img_noise, img_denoise)
    plot_hists(img,img_noise,img_denoise)
    print(str(signal_to_noise(img)) + " & " + str(signal_to_noise(img_noise)) + " & " + str(signal_to_noise(img_denoise)))

In [109]:
import cv2
import random
from PIL import Image
noise_imgs = glob('/kaggle/input/9-classes-noisy-image-dataset/dataset/train_im/speckle/*')
random.shuffle(noise_imgs)
for img_path in noise_imgs[:20]:
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    im_pil = Image.fromarray(img)
    original_image, output_image = inferer.infer_streamlit(im_pil)
    plot_result(original_image,original_image, output_image)

In [81]:
import cv2
import random
from PIL import Image
noise_imgs = glob('/kaggle/input/natural-images-with-synthetic-noise/train/train/noisy images/speckle*_c*')
random.shuffle(noise_imgs)
for img_path in noise_imgs[:20]:
    img = cv2.imread(img_path)
    im_pil = Image.fromarray(img)
    original_image, output_image = inferer.infer_streamlit(im_pil)
    plot_result(original_image, original_image, output_image)